![alt text](img/LM.png)
# Kurs: Deep Learning, Text Mining i XAI w Pythonie

## Autor: Piotr Ćwiakowski

### Lekcja 6. NLP: Uczenie nadzorowane i nienadzorowane

# Zadanie 1. 

Wczytaj zbiór `Womens Clothing E-Commerce Reviews.csv` i następnie wykonaj poniższe polecenia:

- policz liczbę słów w każdej recenzji (i wyniki zaprezentuj za pomocą histogramu),
- sentyment w każdej recenzji (i wyniki zaprezentuj za pomocą histogramu),
- prawdopodobieństwo pozytywnej recenzji (i wyniki zaprezentuj za pomocą histogramu),
- korelację między ratingiem a prawdopodobieństwem pozytywnej recenzji,

Policz średni sentyment ze względu na Departamenty, Dywizje i Rodzaje ubrań. Czy klienci tak samo są zadowoleni ze wszystkich typów ubrań?

In [11]:
import pandas as pd
from textblob import TextBlob
df = pd.read_csv('dataTM/Womens Clothing E-Commerce Reviews.csv')

df.drop('Unnamed: 0', axis=1, inplace=True)
df.drop('Title', axis=1, inplace=True)
df.set_index('Clothing ID', inplace=True)
df = df[~df['Review Text'].isnull()]

def preprocess(ReviewText):
    ReviewText = ReviewText.str.replace("(<br/>)", "", regex=True)
    ReviewText = ReviewText.str.replace('(<a).*(>).*(</a>)', '', regex=True)
    ReviewText = ReviewText.str.replace('(&amp)', '', regex=True)
    ReviewText = ReviewText.str.replace('(&gt)', '', regex=True)
    ReviewText = ReviewText.str.replace('(&lt)', '', regex=True)
    ReviewText = ReviewText.str.replace('(\xa0)', ' ', regex=True)  
    return ReviewText
df['Review Text'] = preprocess(df['Review Text'])
df.head()

,Age,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
Clothing ID,,,,,,,,
767,33,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1080,34,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
1077,60,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
1049,50,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
847,47,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


Następnie przygotuj Document Term Matrix i policz:

- 20 najpopularniejszych słów (po usunięciu stopwords (i może lematyzacji?)
- 20 najpopularniejszych bigramów.

## Zadanie 2. 

Wczytaj zbiór `paper.p` i wykonaj segmentację danych możesz użyc innej techniki niż KMeans.

# Zadanie 3. 

Wczytaj zbiór `paper.p` i wykonaj analizę tematów metodą LDA.

# Zadanie 4.

Wczytaj zbiór z recenzjami filmów (przygotowany podczas poprzednich zajęć), stwórz DTM i wykonaj model prognozujący czy recenzja jest pozytywna czy negatywna.

# Zadanie 5.

Wczytaj zbiór z recenzjami restauracji, stwórz DTM i wykonaj model prognozujący czy recenzja jest pozytywna czy negatywna.

In [4]:
import pickle
import re
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Text Mining
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer 
from sklearn import metrics

# Machine Learning - supervised
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.ensemble import RandomForestClassifier

from helpers import run_xgb
from helpers2 import plotROCs

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [2]:
import pandas as pd

recenzje = pd.read_csv('data/Restaurant_Reviews.tsv', sep='\t')
recenzje

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
995,I think food should have flavor and texture an...,0
996,Appetite instantly gone.,0
997,Overall I was not impressed and would not go b...,0
998,"The whole experience was underwhelming, and I ...",0


In [3]:
recenzje.Liked.value_counts()

1    500
0    500
Name: Liked, dtype: int64

In [14]:
import re
import string

In [16]:
recenzje = pd.read_csv('data/Restaurant_Reviews.tsv', sep='\t')

lemmatizer = WordNetLemmatizer()
stop_words = stopwords.words('english')

# Feature engineering
recenzje['wordCount'] = recenzje['Review'].apply(lambda x: len(x.split()))
recenzje['sentiment'] = recenzje['Review'].apply(lambda x: TextBlob(x).sentiment.polarity)

# Czyszczenie danych
recenzje['Review'] = recenzje['Review'].apply(lambda x: x.lower())
recenzje['Review'] = recenzje['Review'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
recenzje['Review'] = recenzje['Review'].apply(lambda x: ' '.join([x for x in x.split() if len(x) > 1]))
recenzje['Review'] = recenzje['Review'].apply(lambda x: ' '.join([x for x in x.split()]))
recenzje['Review'] = recenzje['Review'].apply(lambda x: ' '.join([x for x in x.split() if x.isalpha()]))
recenzje['Review'] = recenzje['Review'].apply(lambda x: ' '.join([lemmatizer.lemmatize(x) for x in x.split() if len(x) > 1]))
recenzje['Review'] = recenzje['Review'].apply(lambda x: ' '.join([x for x in x.split() if x not in stop_words]))
recenzje.head()

,Review,Liked,wordCount,sentiment
0,wow loved place,1,4,0.40
1,crust good,0,4,-0.35
2,tasty texture wa nasty,0,8,-1.00
3,stopped late may bank holiday rick steve recom...,1,15,0.20
4,selection menu wa great price,1,12,0.80


In [31]:
recenzje.groupby('Liked').agg({'wordCount': 'mean', 'sentiment':'mean'})

,wordCount,sentiment
Liked,,
0,11.498,-0.119531
1,10.290,0.428052


In [17]:
# Transformacja tabeli
vectorizer = CountVectorizer()
doc_vec = vectorizer.fit_transform(recenzje['Review'])

dfCount = pd.DataFrame(doc_vec.toarray(), columns=vectorizer.get_feature_names_out())
dfTf = dfCount.apply(lambda x: x/x.sum(), axis=1)
dfTf.head()

,absolute,absolutely,absolutley,accident,accommodation,accomodate,accordingly,accountant,ache,acknowledged,...,yelpers,yet,youd,youll,youre,yucky,yukon,yum,yummy,zero
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
features = list(vectorizer.get_feature_names_out())
features2 = ['wordCount', 'sentiment']
target = 'Liked'

In [25]:
df = pd.concat([recenzje[[target]], recenzje[['wordCount', 'sentiment']], dfTf], axis=1)
print(df.shape)
df.dropna(inplace=True)
print(df.shape)
df.head()

(1000, 1810)
(1000, 1810)


,Liked,wordCount,sentiment,absolute,absolutely,absolutley,accident,accommodation,accomodate,accordingly,...,yelpers,yet,youd,youll,youre,yucky,yukon,yum,yummy,zero
0,1,4,0.40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,4,-0.35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,8,-1.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,15,0.20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,12,0.80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
# Przygotujmy wrapper
def CVTestRFClass(features, target, nFolds = 5, randomState=2020, debug=False, *args, **kwargs):
    kf = KFold(n_splits=nFolds, shuffle=True, random_state=randomState)

    # listy do przechowywania wyników:
    testResults = []
    trainResults = []
    predictions = []
    indices = []

    # Pętla walidująca model na kolejnych foldach
    for train, test in kf.split(df.index.values):
        # Przygotowanie estymatora
        clf = RandomForestClassifier(*args, **kwargs, random_state=randomState, n_jobs=-1)
        if debug:
            print(clf)
        # Trenowanie modelu
        clf.fit(df.iloc[train][features], df.iloc[train][target])

        # Przygotowanie prognoz dla zbioru treningowego i testowego
        # UWAGA Sklearn zwracał będzie dwie kolumny prawdopodobieństw dla obydwu klas
        predsTrain = clf.predict_proba(df.iloc[train][features])[:,1]
        preds = clf.predict_proba(df.iloc[test][features])[:,1]
        
        # Zachowajmy informacje o predykcjach dla tego foldu
        predictions.append(preds.tolist().copy())
        
        # Razem z indeksami w oryginalnym data frame
        indices.append(df.iloc[test].index.tolist().copy())
        
        # Policzenie dopasowania za pomocą metryki ROC-AUC
        trainScore = roc_auc_score(df[target].iloc[train], predsTrain)
        testScore = roc_auc_score(df[target].iloc[test], preds)
        
        # Zapisanie wyników do listy 
        trainResults.append(trainScore)
        testResults.append(testScore)
        
        # Informowanie o każdym foldzie razem z wynikami treningowymi możemy opcjonalnie wyświetlać w trakcie
        if debug:
            print("Train AUC:", trainScore,
                  "Valid AUC:", testScore)
        
    return np.round(trainResults,5), np.round(testResults,5), predictions, indices

In [27]:
trainResults, testResults, predictions, indices = CVTestRFClass(features=features, target=target, max_depth=12)
print(np.mean(trainResults), np.mean(testResults))

0.974458 0.8286399999999998


In [28]:
trainResults, testResults, predictions, indices = CVTestRFClass(features=features, target=target, max_depth=10)
print(np.mean(trainResults), np.mean(testResults))

0.9670799999999999 0.8298719999999999


In [29]:
trainResults, testResults, predictions, indices = CVTestRFClass(features=features+features2, target=target, max_depth=10)
print(np.mean(trainResults), np.mean(testResults))

0.9598660000000001 0.87456


In [32]:
trainResults, testResults, predictions, indices = CVTestRFClass(features=features+features2, target=target, max_depth=10,
                                                               min_samples_split=30)
print(np.mean(trainResults), np.mean(testResults))

0.948772 0.8738020000000001


In [33]:
trainResults, testResults, predictions, indices = CVTestRFClass(features=features+features2, target=target, max_depth=10,
                                                               min_samples_split=50)
print(np.mean(trainResults), np.mean(testResults))

0.946004 0.8737440000000001


In [34]:
trainResults, testResults, predictions, indices = CVTestRFClass(features=features+features2, target=target, max_depth=10,
                                                               min_samples_split=80)
print(np.mean(trainResults), np.mean(testResults))

0.945826 0.8735839999999999


In [36]:
trainResults, testResults, predictions, indices = CVTestRFClass(features=features+features2, target=target, max_depth=10,
                                                               min_samples_split=80, min_samples_leaf=5)
print(np.mean(trainResults), np.mean(testResults))

0.918342 0.86029


In [37]:
trainResults, testResults, predictions, indices = CVTestRFClass(features=features+features2, target=target, max_depth=10,
                                                               min_samples_split=80, min_samples_leaf=3)
print(np.mean(trainResults), np.mean(testResults))

0.9357620000000001 0.868516


In [38]:
# Przygotujmy prosty wrapper
def CVTestXGB(features, target, nFolds = 5, randomState=2020, debug=False, *args, **kwargs):
    
    # Przygotujmy walidację krzyżową
    kf = KFold(n_splits=nFolds, shuffle=True, random_state=randomState)

    # Zapisanie wyników ze zbioru treningowego i testowego
    testResults = []
    trainResults = []
    
    # Przechowajmy również predykcje dla poszczególnych foldów
    predictions = []
    
    # Razem z informacją o tym, jaki był ich indeks w oryginalnym zbiorze danych
    indices = []
    
    # Przygotujmy listę historii
    hists = []
    fold = 1
    for train, test in kf.split(df.index.values):
        # Przygotowanie estymatora
        score, trainPred, testPred, train_history, impFig, imp = run_xgb(df.iloc[train], df.iloc[test],
                                                                         features, target, debug=debug, *args, **kwargs)

        # Zachowajmy informacje o predykcjach dla tego foldu
        predictions.append(testPred.tolist().copy())
        
        # Razem z indeksami w oryginalnym data frame
        indices.append(df.iloc[test].index.tolist().copy())
        
        # Informowanie o każdym foldzie razem z wynikami treningowymi możemy opcjonalnie wyświetlać w trakcie
        trainScore = roc_auc_score(df[target].iloc[train], trainPred)
        testScore = roc_auc_score(df[target].iloc[test], testPred)
        
        trainResults.append(trainScore)
        testResults.append(testScore)
        
        hists.append(train_history.add_suffix('_'+str(fold)))
        fold+=1
        
        if debug:
            print("Train AUC:", trainScore,
                  "Valid AUC:", testScore)
        
    return trainResults, testResults, predictions, indices, pd.concat(hists, axis=1)

In [39]:
trainResults, testResults, predictions, indices, hists = CVTestXGB(features+features2, target, max_depth=7, subsample = 0.8, num_boost_round=300, 
                                                                   colsample_bytree = 0.7, lambdaX=1, gamma=6, debug=False)
print(np.mean(trainResults), np.mean(testResults))

0.8945900172543396 0.8744071649638281


In [40]:
trainResults, testResults, predictions, indices, hists = CVTestXGB(features+features2, target, max_depth=7, subsample = 0.8, num_boost_round=300, 
                                                                   colsample_bytree = 0.7, lambdaX=1, gamma=6, debug=False, eta=.1)
print(np.mean(trainResults), np.mean(testResults))

0.9007885983595442 0.8730128077772701
